In [1]:
#Lecture 14
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple

In [2]:
class planet():
    "A planet in out solar system"
    def __init__(self,semimajor,eccentricity):
        self.x   = np.array(2) #x and y position
        self.v   = np.array(2) #x and y velocity
        self.a_g = np.array(2) #x and y acceleration
        self.t  = 0.0          #current time
        self.dt = 0.0          #current timestep
        self.a = semimajor     #semimajor axis of the orbit
        self.e = eccentricity  #eccentricity of the orbit
        self.istep = 0         #current integer timestep

In [3]:
solar_system = {"M_sun":1.0, "G":39.4784176043574320}

In [4]:
def solar_circular_velocity(p,solar_system):
    G = solar_system["G"]
    M = solar_system["M_sun"]
    r = (p.x[0]**2+p.x[1]**2)**0.5
    
    #acceleration in AU/yr^2
    a_grav = -1.0*G*M/r**2
    
    #find the angle at this position
    if(p.x[0]==0.0):
        if(p.x[1]>0.0):
            theta = 0.5*np.pi
        else:
            theta = 1.5*np.pi   
    else:
        theta = np.atan(p.x[1],p.x[0])
        
    #set the x and y components of the acceleration
    p.a_g[0] = a_grav *np.cos(theta)
    p.a_g[1] = a_grav *np.sin(theta)
    
    #return the circular velocity AU/year
    return (G*M/r)**0.5

In [5]:
def calc_dt(p):
    
    #integration toler
    ETA_TIME_STEP = 0.0004
    
    #compute timestep
    eta = ETA_TIME_STEP    
    v = (p.v[0]**2 + p.v[1]**2)**0.5
    a = (p.a_g[0]**2 + p.a_g[1]**2)**0.5
    dt = ets * np.fmin(1./np.fabs(v),1./np.sqrt(a))
    
    return dt

In [6]:
def SetPlanet(p,i):
    
    AU_in_km = 1.495979e+8 #an AU in km
    
    #circular velocity
    v_c = 0.0  #circular velocity in AU/yt
    v_a = 0.0  #velocity in perihelion in AU/yr
    
    #planet-by planet initial conditions
    
    #Mercury
    if(i==0):
        #semi-major axis in AU
        p.a = 57909227.0 #AU_in_km
        
        #eccentricity
        p.e = 0.20563593
    
    #Venus
    elif(i==1):
        #semi-major axis in AU
        p.a = 108209475.0 #AU_in_km
        
        #eccentricity
        p.e = 0.00677672
        
    #Earth
    elif(i==2):
        #semi-major axis in AU
        p.a = 1.0
        
        #eccentricity
        p.e = 0.01671123
        
    #set remaining properties
    p.t = 0.0
    p.x[0] = p.a*(1.0-p.e)
    p.x[1] = 0.0
    
    #get equiv circular velocity
    v_c = solar_circular_velocity(p)
    
    #velocity at perihelion
    v_e = v_c(1 + p.e)**0.5
    
    #set velocity
    p.v[0] = 0.0   #no x velocity at perihelion
    p.v[1] = v_e   #y velocity at perihelion (counter clockwise)
    
    #calculate gravitational acceleration from Sun
    
    #set timestep
    p.dt = calc_dt(p)
        

In [8]:
def x_first_step(x_i, v_i, a_i, dt):
    #x_1/2 = x_0 + 1/2 v_0 Delta_t + 1/4 a_0 Delta_t^2
    return x_i + 0.5*v_i*dt + 0.25*a_i*dt**2

In [9]:
def v_full_step(x_i, v_i, a_ipoh, dt):
    #v_i+1 = v_i + a_i+1/2 Delta_t
    return v_i + a_ipoh * dt

In [10]:
def x_full_step(x_ipoh, v_ipl, a_ipoh, dt):
    #x_3/2 = x_1/2 + v_i+1 Delta_t
    return x_ipoh + v_ipl*dt